In [36]:
from mlsocket import MLSocket
import numpy as np
import cv2
from contextlib import contextmanager
import time
from imutils.video import FileVideoStream
import json
from tqdm.notebook import tqdm
import glob
import os
import signal

class Timeout(Exception):
    pass

def handler(sig, frame):
    raise Timeout

def yield_images_from_video(video_path):
    v_cap = FileVideoStream(video_path).start()
    v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))

    for j in range(v_len):
        # get video frame
        frame = v_cap.read()

        if frame is not None:
            img_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            yield img_RGB

signal.signal(signal.SIGALRM, handler)  # register interest in SIGALRM events


HOST = ''
PORT = 27004

In [2]:
vidpaths = sorted(glob.glob('/home/tk/datasets/MELD/MELD.Raw/*/*/*.mp4'))
rootdir = '/home/tk/datasets/MELD/visual-features/box-probs-coarse-landmarks'

already_done = []
print(f"the number of videos to go through: {len(vidpaths)}")
existing = glob.glob(os.path.join(rootdir, '*', '*.json'))
for exist in existing:
    diautt = exist.split('/')[-1].split('.json')[0]
    dataset = exist.split('/')[-2]
    for vp in vidpaths:

        diautt_ = vp.split('/')[-1].split('.mp4')[0]
        dataset_ = vp.split('/')[-3]

        if diautt == diautt_ and dataset in dataset_:
            already_done.append(vp)
            vidpaths.remove(vp)

print(f"some are already done")
print(f"the number of videos to go through: {len(vidpaths)}")

the number of videos to go through: 13848
some are already done
the number of videos to go through: 11441


In [3]:
with MLSocket() as s:

    s.connect((HOST, PORT))  # Connect to the port and host

    for vidpath in tqdm(vidpaths):

        dataset = vidpath.split('/')[-3]
        filename = vidpath.split('/')[-1].replace('.mp4', '.json')
        fullpath = os.path.join(rootdir, dataset, filename)


        to_dump = {'boxes': {}, 'probs': {}, 'landmarks': {}}
        for idx, img in enumerate(yield_images_from_video(vidpath)):
            try:
                # start = time.time()

                img_h, img_w, _ = np.shape(img)

                signal.alarm(10)  # timeout in 10 seconds
                s.send(img)

                boxes = s.recv(1024)
                probs = s.recv(1024)
                landmarks = s.recv(1024)

                to_dump['boxes'][idx] = boxes.tolist()
                to_dump['probs'][idx] = probs.tolist()
                to_dump['landmarks'][idx] = landmarks.tolist()

                # end = time.time()
                # print(f"{1/(end-start)} fps")
            except Exception as e:
                print(e)
                print(f"something wrong with the file {vidpath}, frame number {idx}")
                pass

        with open(fullpath, 'w')  as stream:
            json.dump(to_dump, stream)



something wrong with the file /home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/dia220_utt0.mp4, frame number 287

something wrong with the file /home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/dia229_utt8.mp4, frame number 46

something wrong with the file /home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/dia244_utt6.mp4, frame number 3

something wrong with the file /home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/dia262_utt16.mp4, frame number 2

something wrong with the file /home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/dia30_utt5.mp4, frame number 18

something wrong with the file /home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/dia37_utt3.mp4, frame number 7

something wrong with the file /home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/dia37_utt3.mp4, frame number 23

something wrong with the file /home/tk/datasets/MELD/MELD.Raw/test/output_repeated_splits_test/dia38

In [109]:
import random
from glob import glob
import os
import json
from imutils.video import FileVideoStream


random_vid = random.choice(glob('/home/tk/datasets/MELD/MELD.Raw/train/train_splits/*.mp4'))

prediction_ = '/home/tk/datasets/MELD/visual-features/box-probs-coarse-landmarks/train/' + random_vid.split('/')[-1].replace('.mp4', '.json')

with open(prediction_, 'r') as stream:
    pred = json.load(stream)


images_all = []

for img in yield_images_from_video(random_vid):
    images_all.append(img)

print(f'number of images in the vid: {len(images_all)}')
print(f'number of images in the json: {len(pred["boxes"])}')
print(f'number of images in the json: {len(pred["probs"])}')
print(f'number of images in the json: {len(pred["landmarks"])}')

number of images in the vid: 62
number of images in the json: 62
number of images in the json: 62
number of images in the json: 62
